<a href="https://colab.research.google.com/github/MalakhovDenis/DLS-DL2/blob/main/11.%20%D0%94%D0%97.%20%D0%9F%D1%80%D0%B5%D0%B4%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5%20%D0%B8%20%D0%B4%D0%BE%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5%20%D1%8F%D0%B7%D1%8B%D0%BA%D0%BE%D0%B2%D1%8B%D1%85%20%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B5%D0%B9/11.1%20%D0%A2%D1%80%D0%B0%D0%BD%D1%81%D1%84%D0%BE%D1%80%D0%BC%D0%B5%D1%80%D1%8B/Malakhov%5Bhomework4%5DAttention_and_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p style="align: center;"><img src="https://static.tildacdn.com/tild6636-3531-4239-b465-376364646465/Deep_Learning_School.png" width="400"></p>

# Глубокое обучение. Часть 2
# Домашнее задание по теме "Механизм внимания"

Это домашнее задание проходит в формате peer-review. Это означает, что его будут проверять ваши однокурсники. Поэтому пишите разборчивый код, добавляйте комментарии и пишите выводы после проделанной работы.

В этом задании вы будете решать задачу классификации математических задач по темам (многоклассовая классификация) с помощью Transformer.

В качестве датасета возьмем датасет математических задач по разным темам. Нам необходим следующий файл:

[Файл с классами](https://docs.google.com/spreadsheets/d/13YIbphbWc62sfa-bCh8MLQWKizaXbQK9/edit?usp=drive_link&ouid=104379615679964018037&rtpof=true&sd=true)

**Hint:** не перезаписывайте модели, которые вы получите на каждом из этапов этого дз. Они ещё понадобятся.

### Импорт библиотек и предобработка данных

In [ ]:
!pip install tensorboard evaluate transformers datasets pymorphy2 -q

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Импортируем необходимые библиотеки для дальнейшего анализа

import os
import gc
import random
import copy
import pandas as pd
import numpy as np
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

import nltk
import pymorphy2
nltk.download('stopwords')
from nltk.corpus import stopwords

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
import evaluate

import transformers
from transformers import BertTokenizer, BertModel, get_scheduler

sns.set(palette='summer')

print('Все библиотеки импортированы')

In [ ]:
!nvidia-smi
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

In [ ]:
SEED = 42

def seed_torch(seed):
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
  torch.backends.cudnn.benchmark = False
  torch.backends.cudnn.deterministic = True

seed_torch(SEED)

In [ ]:
# Загрузим данные, посмотрим на их структуру

data = pd.read_excel('./data_problems_translated.xlsx', index_col=0)
print(data.shape)
data.head()

In [ ]:
data.columns = ['text','label']

In [ ]:
print(f'Количество уникальных лейблов: {data.label.nunique()}')
data.label.value_counts()

In [ ]:
data['text'] = data['text'].fillna('').astype(str)
data['len_text'] = data.text.apply(lambda x: len(str(x)))
display(data.describe())
data.len_text.hist()
None

In [ ]:
# Проверка на дубликаты в данных

data.shape, data.drop_duplicates().shape, data.drop_duplicates('text').shape

In [ ]:
data.groupby('text').label.nunique().sort_values(ascending=False).reset_index().head()

In [ ]:
data.dtypes, data.isna().sum()

In [ ]:
data[data.text.str.contains("It's okay. It's okay")].drop_duplicates(['text','label'])

Вывод:
* В данных есть дубликаты по задачам, однако каждый из них имеет несколько Тем! Таким образом решение задачи может быть либо задача Multilabel, либо при вычистке дубликатов можно решать задачу Multiclass.
* Длина предложения самого текста задачи от 0 до 2506 симовлов, при 50 и 75 квантиле 179 и 286 соответственно. Таким образом падить будет не очень хорошей идеей, надо фильтрануть наблюдения по длине предложения

---
1) Удалим Дубликаты по конверсии лейблов

In [ ]:
# Заведем ранжировку для каждого из лейблов и возьмем тот, где первого, где меньше количество наблюдений

rang_dict = {k:v for v,k in enumerate(data.label.value_counts().sort_values().index)}
data['rang_label'] = data.label.map(rang_dict)

In [ ]:
(data.sort_values(['text','rang_label']).drop_duplicates('text',keep='first').drop(columns='rang_label').label.value_counts()
,data.sort_values(['text','rang_label']).drop_duplicates('text',keep='last').drop(columns='rang_label').label.value_counts())

In [ ]:
# Заведем ранжировку для каждого из лейблов и возьмем тот, где первого, где меньше количество наблюдений

rang_dict = {k:v for v,k in enumerate(data.label.value_counts().sort_values().index)}
data['rang_label'] = data.label.map(rang_dict)
data = data.sort_values(['text','rang_label']).drop_duplicates('text',keep='first').drop(columns='rang_label')

In [ ]:
from sklearn.model_selection import train_test_split

label_to_ind = {label:ind for ind,label in enumerate(data.label.unique())}
int_to_label = {ind: label for label, ind in label_to_ind.items()}
NUM_CLASSES = len(label_to_ind)

data['label'] = data['label'].map(label_to_ind)

train_data, eval_data = train_test_split(data[['text','label']], stratify=data['label'], test_size=0.15, random_state=SEED, shuffle=True)

---
2) Посмотрим на выбросы по длинам предложений

In [ ]:
# Part of code is from Seminar Attention and Transformers

ENG_STOP_WORDS = set(stopwords.words('english'))
PUNCT_WORD_TOKENIZER = nltk.WordPunctTokenizer() # for preprocess
# for broadening your horizons use it for lemmatization
MORPH_ANALYZER = pymorphy2.MorphAnalyzer()

def preprocess_text(text):
    nums_filtered_text = re.sub(r'[0-9]+', '', text.lower())
    punct_filtered_text = ''.join(
        [ch for ch in nums_filtered_text if ch not in string.punctuation]
    )
    tokens = PUNCT_WORD_TOKENIZER.tokenize(punct_filtered_text)
    filtr_stop_words_tokens = [MORPH_ANALYZER.parse(token)[0].normal_form for token in tokens
                             if token not in ENG_STOP_WORDS]
    norm_tokens = [MORPH_ANALYZER.parse(token)[0].normal_form for token in filtr_stop_words_tokens]

    return f"[CLS] {' '.join(norm_tokens)}"

In [ ]:
# Слелаем расчет на Интерквартильный размах и посчитаем выбросы по длине предложений

from scipy.stats import iqr

data = data[data.len_text>=data.len_text.quantile(0.01)]

print(data.len_text.quantile(0.25)-iqr(data.len_text),
      data.len_text.quantile(0.25),
      data.len_text.quantile(0.5),
      data.len_text.quantile(0.75),
      data.len_text.quantile(0.75)+iqr(data.len_text))

In [ ]:
# Part of code is from Seminar Attention and Transformers
## Changed hug_dataset to my_dataset

MAX_LENGTH = int(data.len_text.quantile(0.75))

class MyDataset(Dataset):
  def __init__(self,
               my_dataset,
               tokenizer,
               device=DEVICE):

    self.text = my_dataset['text'].apply(lambda x: preprocess_text(x)).tolist()
    self.target = my_dataset['label'].tolist()
    self.tokenizer = tokenizer
    self.device = device

  def __getitem__(self, idx):
    prep_text = self.text[idx]
    target = self.target[idx]

    tokenized_text = self.tokenizer(text=prep_text,
                                    padding="max_length",
                                    max_length=MAX_LENGTH,
                                    truncation=True,
                                    return_tensors='pt')

    text_ids = tokenized_text['input_ids'].flatten()
    attention_mask = tokenized_text['attention_mask'].flatten()

    return text_ids, attention_mask, torch.tensor(target, dtype=torch.long)

  def __len__(self):
    return len(self.text)

### Задание 1 (2 балла)

Напишите кастомный класс для модели трансформера для задачи классификации, использующей в качествке backbone какую-то из моделей huggingface.

Т.е. конструктор класса должен принимать на вход название модели и подгружать её из huggingface, а затем использовать в качестве backbone (достаточно возможности использовать в качестве backbone те модели, которые упомянуты в последующих пунктах)

In [ ]:
### This is just an interface example. You may change it if you want.

class TransformerClassificationModel(nn.Module):
    def __init__(base_transformer_model: Union[str, nn.Module]):
        self.backbone = #...
        # YOUR CODE: create additional layers for classfication

    def forward(inputs, ...):
        # YOUR CODE: propagate inputs through the model. Return dict with logits

        outputs = {<YOUR CODE>}
        return # YOUR CODE

### Задание 2 (1 балл)

Напишите функцию заморозки backbone у модели (если необходимо, возвращайте из функции модель)

In [ ]:
def freeze_backbone_function(model: TransformerClassificationModel):
    pass

### Задание 3 (2 балла)

Напишите функцию, которая будет использована для тренировки (дообучения) трансформера (TransformerClassificationModel). Функция должна поддерживать обучение с замороженным и размороженным backbone.

In [ ]:
import copy

def train_transformer(transformer_model, freeze_backbone=True)
    model = copy.copy(transformer_model)
    ### YOUR CODE IS HERE

    return finetuned_model

### Задание 4 (1 балл)

Проверьте вашу функцию из предыдущего пункта, дообучив двумя способами
*cointegrated/rubert-tiny2* из huggingface.

In [ ]:
rubert_tiny_transformer_model = #...
rubert_tiny_finetuned_with_freezed_backbone = train_transformer(rubert_tiny_transformer_model, freeze_backbone=True)

rubert_tiny_transformer_model = #...
rubert_tiny_full_finetuned = train_transformer(rubert_tiny_transformer_model, freeze_backbone=False)

### Задание 5 (1 балл)

Обучите *tbs17/MathBert* (с замороженным backbone и без заморозки), проанализируйте результаты. Сравните скоры с первым заданием. Получилось лучше или нет? Почему?

In [ ]:
### YOUR CODE IS HERE (probably, similar on the previous step)

### Задание 6 (1 балл)

Напишите функцию для отрисовки карт внимания первого слоя для моделей из задания

In [ ]:
def draw_first_layer_attention_maps(attention_head_ids: List, text: str, model: TransformerClassificationModel):
    pass

### Задание 7 (1 балл)

Проведите инференс для всех моделей **ДО ДООБУЧЕНИЯ** на 2-3 текстах из датасета. Посмотрите на головы Attention первого слоя в каждой модели на выбранных текстах (отрисуйте их отдельно).

Попробуйте их проинтерпретировать. Какие связи улавливают карты внимания? (если в модели много голов Attention, то проинтерпретируйте наиболее интересные)

In [ ]:
### YOUR CODE IS HERE

### Задание 8 (1 балл)

Сделайте то же самое для дообученных моделей. Изменились ли карты внимания и связи, которые они улавливают? Почему?

In [ ]:
### YOUR CODE IS HERE